In [1]:
import numpy as np
from chapter1 import *

### **1. 변수 (Variable)**

In [2]:
data = np.array(1.0)
x = Variable(data)      # x는 Variable의 인스턴스

print(f"val: {x.data}, dim: {x.data.ndim}")

val: 1.0, dim: 0


### **2. 함수 (Function)**

In [3]:
x = Variable(np.array(10))
f = Square()    # Function 클래스를 상속
y = f(x)        # 입력으로 사용되는 x는 Variable 인스턴스

print(f"val: {y.data}, type: {type(y)}")

val: 100, type: <class 'chapter1.Variable'>


### 3. **함수 연결 (Composite Function)**

In [7]:
sqr = Square()
exp = Exp()

# 합성 함수를 통해 "계산 그래프" 생성
x = Variable(np.array(0.5))
h1 = sqr(x)
h2 = exp(h1)
y = sqr(h2)

print(f"val: {y.data: .4f}, type: {type(y)}")

val:  1.6487, type: <class 'chapter1.Variable'>


### **4. 수치 미분 (Numerical Differential)**

- 중앙 차분: x + h와 x - h의 값을 이용한 미분 (오차 최소화)
- 정확도: 유효 자릿수 누락으로 인한 오차 발생
- 계산 비용: 편미분 수행 시(변수가 여러 개인 경우), 각 변수를 찾아 연산을 수행해주어야 함

In [3]:
def f(x):
    sqr = Square()
    exp = Exp()

    return sqr(exp(sqr(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x) # 중앙 차분을 이용한 수치 미분
print(dy)

3.29744262933307e-08


### **5. 역전파 (Backward Pass)**

- 수치 미분과 달리, 자신과 관계된 정보만을 계산
- 연쇄 법칙을 이용하여 미분을 수행하기 때문에 한 번의 역전파로 미분값 계산 가능

In [13]:
# 서로 다른 인스턴스를 생성해야만 forward 시 저장한 정보가 겹치지 않음
sqr1 = Square()
exp = Exp()
sqr2 = Square()

# 순전파 (Forward pass)
x = Variable(np.array(0.5))
h1 = sqr1(x)
h2 = exp(h1)
y = sqr2(h2)

# 역전파 (Backward pass)
y.grad = np.array(1.0)  # y의 gradient를 1로 설정 (자기 자신이므로)
h2.grad = sqr1.backward(y.grad)
h1.grad = exp.backward(h2.grad)
x.grad = sqr2.backward(h1.grad)

print(x.grad) # 미분값

3.297442541400256


### 6. **자동 미분 (Auto Gradient)**

In [ ]:
sqr1 = Square()
exp = Exp()
sqr2 = Square()

# 순전파 (Forward pass)
x = Variable(np.array(0.5))
h1 = sqr1(x)
h2 = exp(h1)
y = sqr2(h2)

# 계산 그래프 (출력 -> 입력 방향)
assert y.creator == sqr2
assert y.creator.input == h2
assert y.creator.input.creator == exp
assert y.creator.input.creator.input == h1
assert y.creator.input.creator.input.creator == sqr1
assert y.creator.input.creator.input.creator.input == x

In [ ]:
# 역전파
y.grad = 1.0
y.backward()
print(x.grad)